In [1]:
# Install required packages
!pip install deep-translator
!pip install langdetect  
!pip install transformers
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!python -m spacy download ru_core_news_sm
!python -m spacy download uk_core_news_sm

# Import necessary libraries
import pandas as pd
import re
import unicodedata
import emoji
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from langdetect import detect
from deep_translator import GoogleTranslator
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=fbe1af6b90cc7fe70767b32d59d71c77afe54636dce1ec38ba8f1e890573d1b6
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 82.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 72.3 MB/s eta 0:00:00:00:010:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the pack

2025-04-18 09:04:49.849036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744967090.140511      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744967090.217919      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
import numpy as np
import re
import unicodedata
import emoji
import spacy
import nltk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

from nltk.corpus import stopwords
from langdetect import detect
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss, accuracy_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    get_linear_schedule_with_warmup
)
from datasets import Dataset
import logging

In [4]:
# Add these imports to your existing imports
import random
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

# Download necessary NLTK resources
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download('omw-1.4', quiet=True)

True

In [5]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/final-dataset/NLP - exp(Encoded UTF-8).csv')
print("Training Dataset Shape:", train_df.shape)
print("Training Dataset Columns:", train_df.columns)
print("Training Dataset Head:\n", train_df.head())

# Load test dataset without skipping rows
test_df = pd.read_csv('/kaggle/input/final-dataset/final test.csv')
print("Test Dataset Shape (raw):", test_df.shape)
print("Test Dataset Columns (raw):", test_df.columns)
print("Test Dataset Head (raw):\n", test_df.head())

Training Dataset Shape: (3822, 6)
Training Dataset Columns: Index(['id', 'content', 'lang', 'manipulative', 'techniques', 'trigger_words'], dtype='object')
Training Dataset Head:
                                      id  \
0  0bb0c7fa-101b-4583-a5f9-9d503339141c   
1  7159f802-6f99-4e9d-97bd-6f565a4a0fae   
2  e6a427f1-211f-405f-bd8b-70798458d656   
3  1647a352-4cd3-40f6-bfa1-d87d42e34eea   
4  9c01de00-841f-4b50-9407-104e9ffb03bf   

                                             content lang  manipulative  \
0  Новий огляд мапи DeepState від російського вій...   uk          True   
1  Недавно 95 квартал жёстко поглумился над русск...   ru          True   
2  🤩\nТим часом йде евакуація Бєлгородського авто...   uk          True   
3  В Україні найближчим часом мають намір посилит...   uk         False   
4  Расчёты 122-мм САУ 2С1 "Гвоздика" 132-й бригад...   ru          True   

                             techniques  \
0        ['euphoria' 'loaded_language']   
1  ['loaded_language' 'c

In [6]:
# Check and fix column names if 'content' is missing
if 'content' not in train_df.columns:
    print("Error: 'content' not found in train_df. Please check the CSV file structure.")
    raise KeyError("'content' column missing in train_df")

if 'content' not in test_df.columns:
    print("Adjusting test_df columns because 'content' is missing...")
    # Assume the second column is 'content' if it exists
    if len(test_df.columns) >= 2:
        test_df.columns = ['id', 'content'] 
    else:
        print("Error: test_df has insufficient columns. Expected at least 'id' and 'content'.")
        raise KeyError("'content' column missing or test_df malformed")
    print("Adjusted Test Dataset Columns:", test_df.columns)
    print("Adjusted Test Dataset Head:\n", test_df.head())

# Add step to filter test_df to exactly 5735 rows by removing invalid rows
print("Initial Test Data Shape:", test_df.shape)
print("First few rows of test_df:\n", test_df.head())

# Remove rows where the 'id' column is invalid (e.g., contains the string "id" or is not a proper data row)
# This assumes the invalid row has 'id' as the value in the 'id' column, which is likely a duplicate header
test_df = test_df[test_df['id'] != 'id']

# If there are still more than 5735 rows, keep only the first 5735 valid rows
# This ensures we don't accidentally remove too many rows if there are other invalid entries
if len(test_df) > 5735:
    test_df = test_df.iloc[:5735]
elif len(test_df) < 5735:
    raise ValueError(f"Test data has fewer than 5735 rows after filtering ({len(test_df)} rows). Check test.csv file.")

print("Test Data Shape after filtering:", test_df.shape)

# Verify the row count
if len(test_df) != 5735:
    raise ValueError(f"Test data should have 5735 rows, but found {len(test_df)} rows after filtering. Check test.csv file.")
    
# Text cleaning function
def clean_text(text):
    text = str(text).lower()
    text = unicodedata.normalize("NFKD", text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-zа-яёіїєґ0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply cleaning
train_df["content"] = train_df["content"].apply(clean_text)
test_df["content"] = test_df["content"].apply(clean_text)
print("Cleaned Train Dataset Head:\n", train_df[["content"]].head())
print("Cleaned Test Dataset Head:\n", test_df[["content"]].head())

Adjusting test_df columns because 'content' is missing...
Adjusted Test Dataset Columns: Index(['id', 'content'], dtype='object')
Adjusted Test Dataset Head:
                                      id  \
0                                    id   
1  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba   
2  9b2a61e4-d14e-4ff7-b304-e73d720319bf   
3  f0f1c236-80a8-4d25-b30c-a420a39be632   
4  31ea05ba-2c2b-4b84-aba7-f3cf6841b204   

                                             content  
0                                            content  
1  Они просрали нашу технику, положили кучу людей...  
2  ❗️\nКитай предлагает отдать оккупированные тер...  
3  Сегодня будет ровно 6 месяцев с этого обещания...  
4  ⚡️\nІзраїль вперше у світі збив балістичну рак...  
Initial Test Data Shape: (5736, 2)
First few rows of test_df:
                                      id  \
0                                    id   
1  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba   
2  9b2a61e4-d14e-4ff7-b304-e73d720319bf   
3  f0f1c236-80a8-

In [7]:
# Define manipulation techniques
techniques = [
    "loaded_language", "glittering_generalities", "euphoria", "appeal_to_fear", "fud",
    "bandwagon", "thought_terminating_cliche", "whataboutism", "cherry_picking", "straw_man"
]
num_classes = len(techniques)

# Convert techniques column to multilabel binary format
def techniques_to_binary(tech_str):
    if pd.isna(tech_str):
        return [0] * num_classes
    tech_str = tech_str.strip("[]").replace("'", "").replace('"', '')
    tech_list = [tech.strip() for tech in tech_str.split() if tech.strip()]
    if not tech_list:
        tech_list = [tech.strip() for tech in tech_str.split(',') if tech.strip()]
    return [1 if tech in tech_list else 0 for tech in techniques]

train_df["labels"] = train_df["techniques"].apply(techniques_to_binary)

# Split dataset
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from nltk.corpus import stopwords
import pickle
import numpy as np
from tqdm import tqdm

# Assuming train_df, val_df, test_df are already loaded
# Load ground truth
ground_truth_df = pd.read_csv('/kaggle/input/final-dataset/solution.csv')

# Define the list of techniques
techniques = [
    'loaded_language', 'glittering_generalities', 'euphoria', 'appeal_to_fear',
    'fud', 'bandwagon', 'thought_terminating_cliche', 'whataboutism',
    'cherry_picking', 'straw_man'
]

# Function to preprocess the techniques column and create binary labels
def extract_technique_labels(df, techniques):
    # Initialize a DataFrame with zeros for each technique
    labels = pd.DataFrame(0, index=df.index, columns=techniques)
    
    # Iterate over each row and set 1 for techniques present in the 'techniques' column
    for idx, row in df.iterrows():
        # Handle cases where techniques might be NaN or empty
        if pd.notna(row['techniques']) and row['techniques'] != '':
            # Convert the techniques string to a list
            tech_list = row['techniques'].replace('[', '').replace(']', '').replace("'", "").split()
            for tech in tech_list:
                # Clean up technique name (remove quotes or other characters)
                tech = tech.strip("'").strip()
                if tech in techniques:
                    labels.at[idx, tech] = 1
    return labels

# Extract labels for train, validation, and test sets
train_labels = extract_technique_labels(train_df, techniques)
val_labels = extract_technique_labels(val_df, techniques)

# Check for single-class columns in train_labels
valid_techniques = []
for tech in techniques:
    unique_classes = train_labels[tech].nunique()
    if unique_classes > 1:
        valid_techniques.append(tech)
    else:
        print(f"Skipping technique '{tech}' due to single class (all {train_labels[tech].unique()[0]})")

# If no valid techniques are found, raise an error
if not valid_techniques:
    raise ValueError("No techniques have sufficient class variation for training.")

# Filter labels to include only valid techniques
train_labels_filtered = train_labels[valid_techniques]
val_labels_filtered = val_labels[valid_techniques]

# Map ground truth columns
gt_columns = [
    'id', 'straw_man', 'appeal_to_fear', 'fud', 'bandwagon', 'whataboutism',
    'loaded_language', 'glittering_generalities', 'euphoria', 'cherry_picking', 'cliche'
]
column_mapping = {
    'straw_man': 'straw_man', 'appeal_to_fear': 'appeal_to_fear', 'fud': 'fud',
    'bandwagon': 'bandwagon', 'whataboutism': 'whataboutism', 'loaded_language': 'loaded_language',
    'glittering_generalities': 'glittering_generalities', 'euphoria': 'euphoria',
    'cherry_picking': 'cherry_picking', 'cliche': 'thought_terminating_cliche'
}
ground_truth_df = ground_truth_df.set_index('id').loc[test_df['id']].reset_index()
ground_truth_labels = ground_truth_df.rename(columns=column_mapping)[techniques].values.astype(int)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words=stopwords.words('english'))
X_train = vectorizer.fit_transform(train_df['content'])
X_val = vectorizer.transform(val_df['content'])
X_test = vectorizer.transform(test_df['content'])

# Train MultiOutputClassifier with Logistic Regression on valid techniques
classifier = MultiOutputClassifier(LogisticRegression(class_weight='balanced', max_iter=1000))
classifier.fit(X_train, train_labels_filtered)

# Evaluate on validation set
val_preds_filtered = classifier.predict(X_val)
print("Validation Set Evaluation Metrics (Filtered Techniques):")
print(f"Macro F1: {f1_score(val_labels_filtered, val_preds_filtered, average='macro', zero_division=0):.4f}")
print("\nClassification Report:\n", classification_report(val_labels_filtered, val_preds_filtered, target_names=valid_techniques, zero_division=0))

# Predict on test set for valid techniques
test_preds_filtered = classifier.predict(X_test)

# Create full predictions array, filling invalid techniques with zeros
test_preds = np.zeros((test_preds_filtered.shape[0], len(techniques)), dtype=int)
for i, tech in enumerate(valid_techniques):
    tech_idx = techniques.index(tech)
    test_preds[:, tech_idx] = test_preds_filtered[:, i]

# Create submission DataFrame
submission_df = pd.DataFrame(test_preds, columns=techniques)
submission_df.insert(0, "id", test_df['id'].tolist())

# Reorder and rename for submission
submission_columns = [
    "id", "straw_man", "appeal_to_fear", "fud", "bandwagon", "whataboutism",
    "loaded_language", "glittering_generalities", "euphoria", "cherry_picking", "cliche"
]
submission_df = submission_df.rename(columns={'thought_terminating_cliche': 'cliche'})
submission_df = submission_df[submission_columns]

# Save submission
submission_df.to_csv("submission_ml.csv", index=False)
print("\nSubmission saved as 'submission_ml.csv'")

# Evaluate on test set with ground truth
test_metrics = classification_report(ground_truth_labels, test_preds, target_names=techniques, zero_division=0)
print("\nTest Set Evaluation Metrics:")
print(f"Macro F1: {f1_score(ground_truth_labels, test_preds, average='macro', zero_division=0):.4f}")
print("\nClassification Report:\n", test_metrics)

# Save model and vectorizer
with open('ml_model.pkl', 'wb') as f:
    pickle.dump(classifier, f)
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
print("Model and vectorizer saved.")

Skipping technique 'thought_terminating_cliche' due to single class (all 0)
Validation Set Evaluation Metrics (Filtered Techniques):
Macro F1: 0.3814

Classification Report:
                          precision    recall  f1-score   support

        loaded_language       0.73      0.68      0.70       392
glittering_generalities       0.50      0.65      0.56        97
               euphoria       0.39      0.60      0.47        77
         appeal_to_fear       0.26      0.41      0.32        58
                    fud       0.33      0.53      0.40        75
              bandwagon       0.14      0.14      0.14        35
           whataboutism       0.13      0.24      0.17        34
         cherry_picking       0.38      0.68      0.49        97
              straw_man       0.13      0.28      0.18        25

              micro avg       0.46      0.59      0.51       890
              macro avg       0.33      0.47      0.38       890
           weighted avg       0.51      0.5